## Flow
## 1. Remove words after (number)
## 2. Merge rows if there is not any (number) until (number) found
## 3. Extract the marks


In [2]:
!pip install PyPDF2

^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import PyPDF2
import re

def extract_questions_from_pdf(pdf_path):
    questions = []
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        pattern = re.compile(r'^\d+\s+')

        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text()
            lines = text.split('\n')
            current_question = ""
            for line in lines:
                if not line.strip() or "Exam Date & Time:" in line or "https://  " in line or re.match(r"\d+\s+of\s+\d+\s+\d+/\d+/\d+,\s+\d+:\d+\s+[AP]M", line):
                    continue
                if pattern.match(line):
                    if current_question:
                        questions.append(current_question.strip())
                    current_question = line.strip()
                else:
                    current_question += " " + line.strip()
            if current_question:
                questions.append(current_question.strip())

    return questions

# Extract questions from the PDF
questions = extract_questions_from_pdf('/content/uni nov 2023.pdf')



# Filter out only the questions that start with an integer
filtered_questions = [question for question in questions if question[0].isdigit()]

# Create a dataframe
df = pd.DataFrame(filtered_questions, columns=['Questions'])

# Reset index starting from 1
df.index = df.index + 1

# Print the dataframe
print(df)
df.to_csv('/content/extracted_questions_daa.csv', index_label='Question_Number')


                                            Questions
1   1 Given two algorithms, Algorithm X and Algori...
2   2 Arrange the following functions in ascending...
3   3 What is the time complexity of the recursive...
4   4 Consider a minimization problem where findin...
5   5 Which of the following is the correct equati...
6   6 Suppose you have candidate set C with coins ...
7   7 State true or false: Kruskal's algorithm can...
8   8 State True or false: The Bellman-Ford algori...
9   9 Choose the correct option for the following ...
10  10 Assume that the algorithms considered here ...
11  11 Let X be a problem that belongs to the clas...
12  12 State true or false: In huffman coding, The...
13  13 How many spanning trees does the given grap...
14  14 How many comparisons are needed to sort an ...
15  15 Consider the below table for jobs given wit...
16  16 What is the basic principle in Rabin Karp a...
17  17 What is recurrence and time complexity for ...
18  18 State Master Theorem.

In [7]:
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

In [8]:
df=pd.read_csv(r"/content/extracted_questions_daa.csv",index_col=0)


In [9]:
df

,Questions
Question_Number,
1,"1 Given two algorithms, Algorithm X and Algorithm Y, with timecomplexities O(2^n) and O(n!), respectively, which of the followingstatements is true? 1) Algorithm Y is more efficient than Algorithm X for small input sizes, but Algorithm X becomes more efficient for larger input sizes.2) Algorithm X is more efficient than Algorithm Y for all input sizes.3) Algorithm Y is more efficient than Algorithm X for all input sizes.4) Algorithm X is more efficient than Algorithm Y for small input sizes, but Algorithm Y becomes more efficient for larger input sizes.(1)"
2,"2 Arrange the following functions in ascending order of theirgrowth. F1(n)=2n, F2(n)=n3/2,F3(n)=nlog 2n, F4(n)=nlog2n 1) F3, F2, F1, F4 2) F2, F3, F1, F4 3) F2, F3, F4, F1 4) F3, F2, F4, F1(2)"
3,3 What is the time complexity of the recursive implementation used tofind the nth Fibonacci term? 1) Linear 2) Polynomial 3) Exponential 4) None of the above(1)
4,4 Consider a minimization problem where finding the optimal solutionis computationally infeasible. Which of the following statementsabout approximation algorithms is true? 1) Approximation algorithms always guarantee the optimal solution.2) Approximation algorithms provide a solution that is arbitrarily close to the optimal solution.3) Approximation algorithms trade optimality for efficiency by providing a solution that is guaranteed to be within a certain factor of the optimal solution.4) Approximation algorithms are only used for maximization problems.(1)
5,"5 Which of the following is the correct equation for the matrix chainmultiplication problem where mat[i-1] * mat[i] gives the dimensionof the ith matrix? 1) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}2) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j].3) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}4) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j](1)"
6,"6 Suppose you have candidate set C with coins of differentdenominations and you want to find the change of an amount N. Youhave an infinite supply of each of coins in C. According to greedyalgorithm, which of the following options, with values of C and K,will NOT produce an optimal answer? 1) C=(1,3,4} and N=6 2) C=(1,4,9} and N=10 3) C=(1,3,8} and N=12 4) C=(1,3,4} and N=100(1)"
7,7 State true or false: Kruskal's algorithm can work for undirectedgraph only for finding MST. 1) true 2) false(1)
8,"8 State True or false: The Bellman-Ford algorithm indicates whetherthere is a negative-weight cycle that is reachable from the source.If there is such a cycle, the algorithm indicates that no solutionexists. 1) true 2) false(1)"
9,9 Choose the correct option for the following table: Algorithm Design approach A. Huffman coding i. Dynamic Programming B.Bellman Ford ii. Backtracking(1)Firefox https://examcloud.in/epn/reports/exam-qpaper.php


## 1. Remove words after (number)

In [10]:
def remove_text_after_number(text):
    pattern = r'\((\d+)\)'  # Regular expression pattern to match a number in parentheses
    match = re.search(pattern, text)

    if match:
        end_index = match.end()  # Get the index of the closing parenthesis
        return text[:end_index]  # Return the part of the string before the closing parenthesis
    else:
        return text

In [11]:
df['Questions'] = df['Questions'].apply(lambda x: remove_text_after_number(x))

In [12]:
df

,Questions
Question_Number,
1,"1 Given two algorithms, Algorithm X and Algorithm Y, with timecomplexities O(2^n) and O(n!), respectively, which of the followingstatements is true? 1) Algorithm Y is more efficient than Algorithm X for small input sizes, but Algorithm X becomes more efficient for larger input sizes.2) Algorithm X is more efficient than Algorithm Y for all input sizes.3) Algorithm Y is more efficient than Algorithm X for all input sizes.4) Algorithm X is more efficient than Algorithm Y for small input sizes, but Algorithm Y becomes more efficient for larger input sizes.(1)"
2,"2 Arrange the following functions in ascending order of theirgrowth. F1(n)=2n, F2(n)=n3/2,F3(n)=nlog 2n, F4(n)=nlog2n 1) F3, F2, F1, F4 2) F2, F3, F1, F4 3) F2, F3, F4, F1 4) F3, F2, F4, F1(2)"
3,3 What is the time complexity of the recursive implementation used tofind the nth Fibonacci term? 1) Linear 2) Polynomial 3) Exponential 4) None of the above(1)
4,4 Consider a minimization problem where finding the optimal solutionis computationally infeasible. Which of the following statementsabout approximation algorithms is true? 1) Approximation algorithms always guarantee the optimal solution.2) Approximation algorithms provide a solution that is arbitrarily close to the optimal solution.3) Approximation algorithms trade optimality for efficiency by providing a solution that is guaranteed to be within a certain factor of the optimal solution.4) Approximation algorithms are only used for maximization problems.(1)
5,"5 Which of the following is the correct equation for the matrix chainmultiplication problem where mat[i-1] * mat[i] gives the dimensionof the ith matrix? 1) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}2) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j].3) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}4) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j](1)"
6,"6 Suppose you have candidate set C with coins of differentdenominations and you want to find the change of an amount N. Youhave an infinite supply of each of coins in C. According to greedyalgorithm, which of the following options, with values of C and K,will NOT produce an optimal answer? 1) C=(1,3,4} and N=6 2) C=(1,4,9} and N=10 3) C=(1,3,8} and N=12 4) C=(1,3,4} and N=100(1)"
7,7 State true or false: Kruskal's algorithm can work for undirectedgraph only for finding MST. 1) true 2) false(1)
8,"8 State True or false: The Bellman-Ford algorithm indicates whetherthere is a negative-weight cycle that is reachable from the source.If there is such a cycle, the algorithm indicates that no solutionexists. 1) true 2) false(1)"
9,9 Choose the correct option for the following table: Algorithm Design approach A. Huffman coding i. Dynamic Programming B.Bellman Ford ii. Backtracking(1)


## 2. Merge rows if there is not any (number) until (number) found


In [13]:
def merge_rows_until_pattern(df, column):

    pattern = r'\(([1-6])\)'
    rows_to_merge = []
    merged_sentences = []

    for index, row in df.iterrows():
        sentence = row[column]

        # Check if the pattern is found in the sentence
        if re.search(pattern, sentence):
            # If we have sentences to merge, combine them and append to merged_sentences
            if rows_to_merge:
                merged_sentences.append(' '.join(rows_to_merge + [sentence]))
                rows_to_merge = []  # Reset rows_to_merge
            else:
                merged_sentences.append(sentence)
        else:
            # If the pattern is not found, add the sentence to rows_to_merge
            rows_to_merge.append(sentence)

    # If there are any remaining rows in rows_to_merge, combine and append
    if rows_to_merge:
        merged_sentences.append(' '.join(rows_to_merge))

    # Create a new DataFrame with the merged sentences
    merged_df = pd.DataFrame({column: merged_sentences})
    return merged_df

In [14]:
merged_df = merge_rows_until_pattern(df, 'Questions')


In [15]:
merged_df


,Questions
0,"1 Given two algorithms, Algorithm X and Algorithm Y, with timecomplexities O(2^n) and O(n!), respectively, which of the followingstatements is true? 1) Algorithm Y is more efficient than Algorithm X for small input sizes, but Algorithm X becomes more efficient for larger input sizes.2) Algorithm X is more efficient than Algorithm Y for all input sizes.3) Algorithm Y is more efficient than Algorithm X for all input sizes.4) Algorithm X is more efficient than Algorithm Y for small input sizes, but Algorithm Y becomes more efficient for larger input sizes.(1)"
1,"2 Arrange the following functions in ascending order of theirgrowth. F1(n)=2n, F2(n)=n3/2,F3(n)=nlog 2n, F4(n)=nlog2n 1) F3, F2, F1, F4 2) F2, F3, F1, F4 3) F2, F3, F4, F1 4) F3, F2, F4, F1(2)"
2,3 What is the time complexity of the recursive implementation used tofind the nth Fibonacci term? 1) Linear 2) Polynomial 3) Exponential 4) None of the above(1)
3,4 Consider a minimization problem where finding the optimal solutionis computationally infeasible. Which of the following statementsabout approximation algorithms is true? 1) Approximation algorithms always guarantee the optimal solution.2) Approximation algorithms provide a solution that is arbitrarily close to the optimal solution.3) Approximation algorithms trade optimality for efficiency by providing a solution that is guaranteed to be within a certain factor of the optimal solution.4) Approximation algorithms are only used for maximization problems.(1)
4,"5 Which of the following is the correct equation for the matrix chainmultiplication problem where mat[i-1] * mat[i] gives the dimensionof the ith matrix? 1) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}2) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j].3) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}4) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j](1)"
5,"6 Suppose you have candidate set C with coins of differentdenominations and you want to find the change of an amount N. Youhave an infinite supply of each of coins in C. According to greedyalgorithm, which of the following options, with values of C and K,will NOT produce an optimal answer? 1) C=(1,3,4} and N=6 2) C=(1,4,9} and N=10 3) C=(1,3,8} and N=12 4) C=(1,3,4} and N=100(1)"
6,7 State true or false: Kruskal's algorithm can work for undirectedgraph only for finding MST. 1) true 2) false(1)
7,"8 State True or false: The Bellman-Ford algorithm indicates whetherthere is a negative-weight cycle that is reachable from the source.If there is such a cycle, the algorithm indicates that no solutionexists. 1) true 2) false(1)"
8,9 Choose the correct option for the following table: Algorithm Design approach A. Huffman coding i. Dynamic Programming B.Bellman Ford ii. Backtracking(1)
9,"10 Assume that the algorithms considered here sort the input sequencesin ascending order. If the input is already in ascending order,which of the following are TRUE? 1) Quicksort runs in Θ(n^2) time2) Bubble sort runs in Θ(n^2) time3) Mergesort runs in Θ(n) time4) Insertion sort runs in Θ(n) time(1)"



## 3. Extract the marks

In [16]:
# Function to extract marks from each row
def extract_marks(row):
    # Using regular expression to find the marks in parentheses
    matches = re.findall(r'\((\d+)\)$', row['Questions'])

    # If matches are found, return the marks
    if matches:
        return int(matches[0])  # Assuming there is only one match, convert to int
    else:
        return None

In [17]:
extracted_marks = pd.DataFrame()
extracted_marks['Marks']= merged_df.apply(extract_marks, axis=1)

In [18]:
extracted_marks

,Marks
0,1
1,2
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [19]:
def remove_last_number(sentence):
    return re.sub(r'\(\d+\)', '', sentence)

In [20]:
merged_df['Questions'] = merged_df['Questions'].apply(remove_last_number)


In [21]:
merged_df

,Questions
0,"1 Given two algorithms, Algorithm X and Algorithm Y, with timecomplexities O(2^n) and O(n!), respectively, which of the followingstatements is true? 1) Algorithm Y is more efficient than Algorithm X for small input sizes, but Algorithm X becomes more efficient for larger input sizes.2) Algorithm X is more efficient than Algorithm Y for all input sizes.3) Algorithm Y is more efficient than Algorithm X for all input sizes.4) Algorithm X is more efficient than Algorithm Y for small input sizes, but Algorithm Y becomes more efficient for larger input sizes."
1,"2 Arrange the following functions in ascending order of theirgrowth. F1(n)=2n, F2(n)=n3/2,F3(n)=nlog 2n, F4(n)=nlog2n 1) F3, F2, F1, F4 2) F2, F3, F1, F4 3) F2, F3, F4, F1 4) F3, F2, F4, F1"
2,3 What is the time complexity of the recursive implementation used tofind the nth Fibonacci term? 1) Linear 2) Polynomial 3) Exponential 4) None of the above
3,4 Consider a minimization problem where finding the optimal solutionis computationally infeasible. Which of the following statementsabout approximation algorithms is true? 1) Approximation algorithms always guarantee the optimal solution.2) Approximation algorithms provide a solution that is arbitrarily close to the optimal solution.3) Approximation algorithms trade optimality for efficiency by providing a solution that is guaranteed to be within a certain factor of the optimal solution.4) Approximation algorithms are only used for maximization problems.
4,"5 Which of the following is the correct equation for the matrix chainmultiplication problem where mat[i-1] * mat[i] gives the dimensionof the ith matrix? 1) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}2) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j].3) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}4) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j]"
5,"6 Suppose you have candidate set C with coins of differentdenominations and you want to find the change of an amount N. Youhave an infinite supply of each of coins in C. According to greedyalgorithm, which of the following options, with values of C and K,will NOT produce an optimal answer? 1) C=(1,3,4} and N=6 2) C=(1,4,9} and N=10 3) C=(1,3,8} and N=12 4) C=(1,3,4} and N=100"
6,7 State true or false: Kruskal's algorithm can work for undirectedgraph only for finding MST. 1) true 2) false
7,"8 State True or false: The Bellman-Ford algorithm indicates whetherthere is a negative-weight cycle that is reachable from the source.If there is such a cycle, the algorithm indicates that no solutionexists. 1) true 2) false"
8,9 Choose the correct option for the following table: Algorithm Design approach A. Huffman coding i. Dynamic Programming B.Bellman Ford ii. Backtracking
9,"10 Assume that the algorithms considered here sort the input sequencesin ascending order. If the input is already in ascending order,which of the following are TRUE? 1) Quicksort runs in Θ(n^2) time2) Bubble sort runs in Θ(n^2) time3) Mergesort runs in Θ(n) time4) Insertion sort runs in Θ(n) time"


In [22]:
final_df=pd.concat([merged_df, extracted_marks], axis=1)

In [23]:
final_df

,Questions,Marks
0,"1 Given two algorithms, Algorithm X and Algorithm Y, with timecomplexities O(2^n) and O(n!), respectively, which of the followingstatements is true? 1) Algorithm Y is more efficient than Algorithm X for small input sizes, but Algorithm X becomes more efficient for larger input sizes.2) Algorithm X is more efficient than Algorithm Y for all input sizes.3) Algorithm Y is more efficient than Algorithm X for all input sizes.4) Algorithm X is more efficient than Algorithm Y for small input sizes, but Algorithm Y becomes more efficient for larger input sizes.",1
1,"2 Arrange the following functions in ascending order of theirgrowth. F1(n)=2n, F2(n)=n3/2,F3(n)=nlog 2n, F4(n)=nlog2n 1) F3, F2, F1, F4 2) F2, F3, F1, F4 3) F2, F3, F4, F1 4) F3, F2, F4, F1",2
2,3 What is the time complexity of the recursive implementation used tofind the nth Fibonacci term? 1) Linear 2) Polynomial 3) Exponential 4) None of the above,1
3,4 Consider a minimization problem where finding the optimal solutionis computationally infeasible. Which of the following statementsabout approximation algorithms is true? 1) Approximation algorithms always guarantee the optimal solution.2) Approximation algorithms provide a solution that is arbitrarily close to the optimal solution.3) Approximation algorithms trade optimality for efficiency by providing a solution that is guaranteed to be within a certain factor of the optimal solution.4) Approximation algorithms are only used for maximization problems.,1
4,"5 Which of the following is the correct equation for the matrix chainmultiplication problem where mat[i-1] * mat[i] gives the dimensionof the ith matrix? 1) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}2) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j].3) dp[i,j] = 0 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]}4) dp[i,j] = 1 if i=j dp[i,j] = min{dp[i,k] + dp[k+1,j]} + mat[i- 1]*mat[k]*mat[j]",1
5,"6 Suppose you have candidate set C with coins of differentdenominations and you want to find the change of an amount N. Youhave an infinite supply of each of coins in C. According to greedyalgorithm, which of the following options, with values of C and K,will NOT produce an optimal answer? 1) C=(1,3,4} and N=6 2) C=(1,4,9} and N=10 3) C=(1,3,8} and N=12 4) C=(1,3,4} and N=100",1
6,7 State true or false: Kruskal's algorithm can work for undirectedgraph only for finding MST. 1) true 2) false,1
7,"8 State True or false: The Bellman-Ford algorithm indicates whetherthere is a negative-weight cycle that is reachable from the source.If there is such a cycle, the algorithm indicates that no solutionexists. 1) true 2) false",1
8,9 Choose the correct option for the following table: Algorithm Design approach A. Huffman coding i. Dynamic Programming B.Bellman Ford ii. Backtracking,1
9,"10 Assume that the algorithms considered here sort the input sequencesin ascending order. If the input is already in ascending order,which of the following are TRUE? 1) Quicksort runs in Θ(n^2) time2) Bubble sort runs in Θ(n^2) time3) Mergesort runs in Θ(n) time4) Insertion sort runs in Θ(n) time",1


In [27]:
final_df.to_csv('final_output.csv',index=False)